In [ ]:
#Loads packages
import numpy as np
import tifffile
import napari
import pandas as pd

from skimage.morphology import ball, binary_dilation, dilation
from skimage.measure import label, regionprops

This notebook is used to quantify interactions between mossy fiber boutons and complex spines. The algorithm requires segmentations of MFBs, pSCRs, and complex spines as input. The output consists of two .csv files containing information about MFB-Spines and Spine-MFBs interaction statistics.

Test data includes exemplary segmentation ROI with two MFBs and a fragment of pyramidal neuron proximal dendrite with multiple complex spines.

#### Load segmentation data

In [ ]:
#Loads segmented MFBs, pSCRs, and PN with complex spines
boutons = tifffile.imread('.\\data\\boutons_roi.tif')
spines_and_pn = tifffile.imread('.\\data\\spines_and_pn_roi.tif')
pscrs = tifffile.imread('.\\data\\pscrs_roi.tif')

In [ ]:
#Creates separate layers for the PN and complex spines
pn_seg = spines_and_pn==85
spines = spines_and_pn-pn_seg*85

In [ ]:
#Sets up napari Viewer and displays segmentation data
viewer = napari.Viewer()
viewer.add_labels(boutons, name='boutons')
viewer.add_labels(spines, name='spines')
viewer.add_labels(pn_seg, name='PN')
viewer.add_labels(pscrs, name='pSCRs')

#### Association of pSCRs with MFBs and complex spines

In [ ]:
#Dilates pSCRs
pscrs_expanded = dilation(pscrs, ball(1))

In [ ]:
#Finds MFBs associated with each individual pSCR
rp_pscrs_boutons = regionprops(pscrs_expanded, intensity_image = boutons)
pscr_stats = pd.DataFrame(columns = ['pscr id', 'list of boutons','overlap in px pscr-bouton'])
for i in range(len(rp_pscrs_boutons)):
    ids_bouton,counts = np.unique(rp_pscrs_boutons[i].intensity_image, return_counts=True)
    counts = counts[np.nonzero(ids_bouton)]
    ids_bouton = ids_bouton[ids_bouton!=0]
    data = [[rp_pscrs_boutons[i].label, list(ids_bouton), counts]]
    df = pd.DataFrame(data=data, columns = ['pscr id', 'list of boutons','overlap in px pscr-bouton'])
    pscr_stats = pscr_stats.append(df)

In [ ]:
#Finds complex spines associated with each individual pSCR
rp_pscrs_spines = regionprops(pscrs_expanded, intensity_image = spines_and_pn)
pscr_stats_1 = pd.DataFrame(columns = ['pscr id', 'list of spines', 'overlap in px pscr-spine'])
for i in range(len(rp_pscrs_spines)):
    ids_spines,counts = np.unique(rp_pscrs_spines[i].intensity_image, return_counts=True)
    counts = counts[np.nonzero(ids_spines)]
    ids_spines = ids_spines[ids_spines!=0]
    data = [[rp_pscrs_spines[i].label, list(ids_spines), counts]]
    df = pd.DataFrame(data=data, columns = ['pscr id', 'list of spines','overlap in px pscr-spine'])
    pscr_stats_1 = pscr_stats_1.append(df)

In [ ]:
#Creates a table containing the results
pscr_stats['list of spines'] = pscr_stats_1['list of spines']
pscr_stats['overlap in px pscr-spine'] = pscr_stats_1['overlap in px pscr-spine']

In [ ]:
#Resolves one-to-many cases
#if one pSCR is associated with more than one bouton, the pair with the largest overlap in voxels is preserved
for i in range(len(pscr_stats)):
    if len(pscr_stats['list of boutons'].iloc[i])>1 and len(pscr_stats['list of spines'].iloc[i])>0:
        pscr_stats['list of boutons'].iloc[i] = [pscr_stats['list of boutons'].iloc[i][np.argmax(pscr_stats['overlap in px pscr-bouton'].iloc[i])]]

#### Get MFB-spines and Spine-MFBs interaction statistics

In [ ]:
#Creates table with MFB-Spines interaction statistics
boutons_stat = pd.DataFrame(columns = ['bouton id', 'num of pscrs', 'num of spines', 'list of spines'])
for i in [11,65]:
    spines_list = []
    num_of_pscrs=0
    for j in range(len(pscr_stats)):
        if i in pscr_stats['list of boutons'].iloc[j]:
            num_of_pscrs+=1
            if len(pscr_stats['list of spines'].iloc[j])>0:
                spines_list.append(pscr_stats['list of spines'].iloc[j])
    spines_list = [item for sublist in spines_list for item in sublist]
    spines_list = list( dict.fromkeys(spines_list) )
    data = [[i, num_of_pscrs,len(spines_list),spines_list]]
    df = pd.DataFrame(data=data, columns = ['bouton id', 'num of pscrs', 'num of spines', 'list of spines'])
    boutons_stat = boutons_stat.append(df)

In [ ]:
#Outputs the MFB-Spines interaction table
boutons_stat

In [ ]:
#Creates table with Spine-MFBs interaction statistics
spine_stat = pd.DataFrame(columns = ['spine id', 'num of pscrs', 'num of boutons', 'list of boutons'])
for i in [7, 10, 11, 15, 21, 22, 27, 28, 31, 44, 45, 51, 59, 62, 66, 67]:
    boutons_list = []
    num_of_pscrs=0
    for j in range(len(pscr_stats)):
        if i in pscr_stats['list of spines'].iloc[j]:
            num_of_pscrs+=1
            if len(pscr_stats['list of boutons'].iloc[j])>0:
                boutons_list.append(pscr_stats['list of boutons'].iloc[j])
    boutons_list = [item for sublist in boutons_list for item in sublist]
    boutons_list = list( dict.fromkeys(boutons_list) )
    data = [[i, num_of_pscrs,len(boutons_list),boutons_list]]
    df = pd.DataFrame(data=data, columns = ['spine id', 'num of pscrs', 'num of boutons', 'list of boutons'])
    spine_stat = spine_stat.append(df)

In [ ]:
#Outputs the Spine-MFBs interaction table
spine_stat

#### Save tables as csv files

In [ ]:
#Saves results
boutons_stat.to_csv('MFB-Spines_interaction_stats.csv')
spine_stat.to_csv('Spines-MFB_interaction_stats.csv')